In [2]:
import pandas as pd
import numpy as np
import multiprocessing
import argparse
from datetime import timedelta
import pyvips
import PIL
import time
import random
import gc
import logging
import os
from pathlib import Path
import sys

sys.path.append('..')
from src.wsi import slides, filters, tiles, util

BASE_PATH = Path(os.path.dirname(os.getcwd()))
DATA_PATH = BASE_PATH / 'data' / 'originals'
PROCESSED_DATA = BASE_PATH / 'data' / 'processed'

In [3]:
train = pd.read_csv(BASE_PATH / 'data' / 'train.csv')
filenames = train['filename'].values.tolist()

In [5]:
train[train['filename'] == 'nv7n6c5c.tif']

,filename,age,sex,melanoma_history,tif_cksum,tif_size,us_tif_url,eu_tif_url,as_tif_url,local_url,relapse
1341,nv7n6c5c.tif,0.897959,0,-1,3070856857,1110554288,s3://drivendata-competition-visiomel-public-us...,s3://drivendata-competition-visiomel-public-eu...,s3://drivendata-competition-visiomel-public-as...,s3://visiomel/originals/nv7n6c5c.tif,0
